In [ ]:
## connects db #from piesagiving_web.config import client


In [ ]:
from restaurant_api.config import client
db = client.restaurant_local

# AttributeError: 'function' object has no attribute 'name'

In [ ]:
@mongoagree.route('mongoAgree') #IF BLUEPRINT NAME AND FUNCTION NAME SAME 
def mongoagree():
    return "Success"

In [ ]:
 #TypeError: Object of type ObjectId is not JSON serializable // Werkzeug Debugger
    #Sollution

In [ ]:
#from bson import json_util  # import 
#return json.dumps(data_status, indent=4, default=json_util.default) # return like this
#Example :
@mongoagree.route('/mongofind')
def mongo():
    data_status={"result":0}   
    cursor = list(db.price_levels.find({}))   
    data_status["result"]=cursor   
    return json.dumps(data_status, indent=4, default=json_util.default)

### Aggregation in Python Code 

In [ ]:
# 
@mongoagree.route('/mongofind')
def mongo():
    data_status={"result":0}    
    cursor = list(db.persons.aggregate([
    {"$match":{"age":{"$gt":39}
        ,"gender":"male","eyeColor":"blue","isActive":False
                }
     }
    ]))
  
    data_status["result"]=cursor
    #return data_status #Object of type ObjectId is not JSON serializable /
    return json.dumps(data_status, indent=4, default=json_util.default)


In [ ]:
## Match and Group

In [ ]:
db.persons.aggregate([
    {$match:{age:{$gt:35},gender:"male",isActive:false,eyeColor:"blue"}},
    {$group:{_id:"$_id",age:{ $first : "$age" },eyeColor:{$first:"$eyeColor"}}}
    ])
"""
Result We Be like
{
	"_id" : ObjectId("5edc9baebca5ba1b08ba4abc"),
	"age" : 37,
	"eyeColor" : "blue"
},

"""

In [ ]:
## Example Match and Group

In [ ]:
db.res_menus.aggregate([
    {$match:{createdBy:ObjectId("5e86cf3e9eb7bc1db005b6a1"),"tareWeight" :11}},
    {$group:{_id:"$uniqueNumber"}}
    ])
"""
/* 1 */
{
	"_id" : "2020050300000010"
},

/* 2 */
{
	"_id" : "2020050300000009"
},
"""

In [ ]:
##Example2

In [ ]:
@mongoagree.route('/agreegation1')
def agreegation1():
    data_status={"result":0}
    data = list(db.res_menus.aggregate([
    {"$match":{"createdBy":ObjectId("5e86cf3e9eb7bc1db005b6a1") }},
    {"$group":{"_id":"$uniqueNumber"}}
    ]))
    data_status["result"]=data
    return json.dumps(data_status,indent=4, default=json_util.default)

In [ ]:
#Filtering Null values

In [ ]:
@mongoagree.route('/agreegation1')
def agreegation1():
    data_status={"result":0}
    data = list(db.menus.aggregate([
    {"$match":{"createdBy":ObjectId("5e86cf3e9eb7bc1db005b52b"),"tareWeight":0 }},
    {"$group":{"_id":"$uniqueNumber"}}
    ]))
    data_status["result"]=data
    return json.dumps(data_status,indent=4, default=json_util.default)
"""
Filtered but Stored 0 to  Specified Null value Object
db.inventory.find( { item : { $exists: false } } ) # its NOt Worked

"""

In [ ]:
##

In [ ]:
db.persons.aggregate([
       {$group:{_id:"$_id",age:{ $last : "$age" },eyeColor:{$first:"$eyeColor"},index:{$first:"$index"}}},
       {$match:{age:25,eyeColor:"blue"}}
       

    ])
## Swap Group and Match   Both Result is Same Here
db.persons.aggregate([
       {$match:{age:25,eyeColor:"blue"}}
        {$group:{_id:"$_id",age:{ $last : "$age" },eyeColor:{$first:"$eyeColor"},index:{$first:"$index"}}},

       

    ])
"""
{
	"_id" : ObjectId("5edc9baebca5ba1b08ba4ad2"),
	"age" : 25,
	"eyeColor" : "blue",
	"index" : 958
},
"""

In [ ]:
## data 

In [ ]:
 countData=list(db.piesa_giving_special_cause_donations.aggregate([
                    {"$match":{"causeId":ObjectId(specialCause_id)}},
                   {"$group": {"_id": None,"totalAmount": {"$sum": "$amount" },
                    "donationsByLength":{"$sum": 1 }}}
                    ]))
                countDict =countData[0]
                del countDict["_id"]
                #print(countData,"*"*20)
                data_status["donationDetails"] = countDict

In [ ]:
# its need for storing image in a folder
SITE_ROOT = os.path.dirname(os.path.realpath(__file__))  # it will give path of current Directory
app.config['SITE_ROOT']=SITE_ROOT


In [ ]:
#BASE 64 string Converting into image and storing in a Folder

In [ ]:
import binascii
from mongoengine import ValidationError
def upload_doc(fileName, purchaseOrderFile,directory ="upload_files"):
	data_status = {"responseStatus": 0, "result": ""}
	try:
		b64 = purchaseOrderFile
		byte = base64.b64decode(b64, validate=True)

		# Write the PDF contents to a local file
		purchase_folder =  os.path.join(app.config['SITE_ROOT'],"media/"+directory )
		if not os.path.exists(purchase_folder):
			os.makedirs(purchase_folder)
            
        #PDF CONVERTING
		if byte[0:4] == b'%PDF':
			mfile = fileName + '.pdf'
			pdf_loc = os.path.join(purchase_folder, mfile)
			f = open(pdf_loc, 'wb')
			f.write(byte)
			f.close()

			if pdf_loc:
				return mfile
			else:
				data_status["responseStatus"]=0
				data_status["result"]="Something went to wrong!"
				return data_status
            #BASE 64 IMAGE CONVERTING
		elif byte[0:4] == b'\xff\xd8\xff\xe0' :
			image_data = bytes(b64, encoding="ascii")
			im = Image.open(BytesIO(base64.b64decode(image_data)))
			rgb_im = im.convert('RGB')
			imageName = fileName + '.png'
			rgb_im.save(os.path.join(purchase_folder, imageName))

			if rgb_im:
				return imageName
			else:
				data_status["responseStatus"]=0
				data_status["result"]="Something went to wrong!"
				return data_status
		else:
			data_status["responseStatus"]=0
			data_status["result"]="Something went to wrong!"
			return data_status
	except binascii.Error as e:
		data_status["responseStatus"]=0
		data_status["result"]="Error!"
		return "Error!"
	except ValidationError as e:
		data_status["responseStatus"]=0
		data_status["result"]="Error!"
		return "Error!"



# Sorting in Mongo

In [ ]:
db.psg_users.find().sort({firstName:-1})  #1-ascending order -1 - discending order
#with Aggregation
db.psg_users.aggregate([
    {$sort:{firstName:-1}}
    ])

# Multiple Sorts
db.psg_users.aggregate([
    {$sort:{firstName:1,email:-1}}
    ])

# Group and Sort

In [ ]:
db.persons.aggregate([
    {$group:{_id:"$favoriteFruit"}},
    {$sort:{_id:1}}
    ])

# Multiple 
db.persons.aggregate([
    {$group:{_id:{eyeColor:"$eyeColor",favoriteFruit:"$favoriteFruit"}}},
    {$sort:{"_id.eyecolor":1,"_id.favoriteFruit":-1}}
    ])
#example 2
db.persons.aggregate([
    // stage 1
    {$match:{eyeColor:{$ne:"blue"}}}, //ne :not euql to
    {$group:{_id:{eyeColor:"$eyeColor",favoriteFruit:"$favoriteFruit"}}},
    {$sort:{"_id.eyecolor":1,"_id.favoriteFruit":-1}}
    ])

# Project Stage -

In [ ]:
# note- project works for each input documents if we have 1000 documents as a input it will get 1000 documents as output 

db.persons.aggregate([
    {$project:{name:1,"company.location.country":1}}
    // stage 1
   
    ])
'''
{
	"_id" : ObjectId("5edc9baebca5ba1b08ba4714"),
	"name" : "Aurelia Gonzales",
	"company" : {
		"location" : {
			"country" : "USA"
		}
	}
}

'''
# except mongo Id 
db.persons.aggregate([
    {$project:{name:1,"company.location.country":1,_id:0}}
    // {$count:"total"}  to know records counts
   
    ])
'''
{
	"name" : "Aurelia Gonzales",
	"company" : {
		"location" : {
			"country" : "USA"
		}
	}
}
'''
#except  these 3 all keys
db.persons.aggregate([
    {$project:{isActive:0,name:0,gender:0}}

    ])

# we can change fields names also # lets create new document struture 
db.persons.aggregate([
    {$project:
        {
            _id:0,
            name:1,
            info:{
                eyes:"$eyeColor",
                fruit:"$favoriteFruit",
                country:"$company.location.country"
            }
        }
        
    }

    ])
'''
{
	"name" : "Aurelia Gonzales",
	"info" : {
		"eyes" : "green",
		"fruit" : "banana",
		"country" : "USA"
	}
},
'''

In [ ]:
# Get  Matched Data in Descending

In [ ]:
db.psg_sign_in_histoty.aggregate([
                            {"$match":{"userId":ObjectId(user_id)}},
                            {"$sort":{"_id":-1}}

                            ])

# Get Date and Time From the  - (ISODate)  -  DateField  

In [ ]:
db.psg_sign_in_histoty.aggregate(
   [
       {"$match":{"userId":ObjectId("5ee30b48251376f63f6a18a8")}},
       {$sort:{_id:-1}},
     {
       $project: {
          _id: 0,
          date: { $dateToString: { format: "%Y-%m-%d", date: "$dateTime" } },
          time: { $dateToString: { format: "%H:%M", date: "$dateTime", timezone: "+05:30"} },
          type:"$type",
          location:"$location",
          status:"$status"         
     }
   ]
)

In [22]:
lst1 = [{"id": 1, "x": "one"}, {"id": 2, "x": "two"}]
lst2 = [{"id": 2, "x": "two"}, {"id": 3, "x": "three"}]  

result = []
lst1.extend(lst2)
for myDict in lst1:
    if myDict not in result:
        result.append(myDict)
print(result)

[{'id': 1, 'x': 'one'}, {'id': 2, 'x': 'two'}, {'id': 3, 'x': 'three'}]


In [32]:
dictlist=[]
lst1 = [{"id": 1, "x": "one"}, {"id": 2, "x": "two"},{"id": 3, "x": "two"},{"id": 4, "x": "two"}]
for dict1 in lst1:
    print(dict1)
    for k,v in dict1.items():
        a ={}
        a["KeyName"]=k
        a["KeyValue"]=v
        dictlist.append(a)               
        
        
    

{'id': 1, 'x': 'one'}
{'id': 2, 'x': 'two'}
{'id': 3, 'x': 'two'}
{'id': 4, 'x': 'two'}
[{'KeyName': 'id', 'KeyValue': 1}, {'KeyName': 'x', 'KeyValue': 'one'}, {'KeyName': 'id', 'KeyValue': 2}, {'KeyName': 'x', 'KeyValue': 'two'}, {'KeyName': 'id', 'KeyValue': 3}, {'KeyName': 'x', 'KeyValue': 'two'}, {'KeyName': 'id', 'KeyValue': 4}, {'KeyName': 'x', 'KeyValue': 'two'}]


In [ ]:
d1  = [{"name":"A","value":1},{"name":"B","value":2},{"name":"C","value":3},{"name":"C","value":"4"}]
d2 = [{"name":"A","value":1},{"name":"B","value":2},{"name":"C","value":3}]
listFinal =[]
for dc2 in d2:
    for dc1 in d1:
       if dc1 not in d2:
          d2.append(d1)
print(d1)
        


In [52]:
import datetime

dateTime=datetime.datetime.now()
TodayDateNo=dateTime.strftime("%j")
TodayDate =dateTime.strftime("%d/%m/%Y")
final =[]
dict1 ={
    "weekDay":"TodayDate"
}
list1 = []
def fun1(n):
    for x in range(7):   
        aa = int(TodayDateNo) -x
        dict1[aa]="aaaa"
        
        
    fun1(n=10)     
print(dict1)
    

    
    
    
    

    
    
    


{'weekDay': 'TodayDate'}


In [43]:

l2= []
Input1 = [{"index":"1", "value":"2"}, {"index":"2", "value":"3"}, {"index":"3", "value":"eggs"},{"index":"4", "value":"eggs"}]
Input2 = [{"index":"1", "value":"4"}, {"index":"2", "value":"5qq"}]
# for ab in Input1:
#     for bc in Input2:
#         if ab["index"] == bc["index"]:
#             Input1.remove(ab)
            


db =[1,2,3]#get
get =[4,3,5]#db
[ d for d in get if d not in db ]

[4, 5]

# Merge List of Dictionaries By specific Common key

In [ ]:
# Merge List of Dictionaries By specific Common key
#Compare Db List and Get list Remove Duplicates
def remove_duplicates(get_list,list_from_db,key_to_filter): 
	get_ids = []
	for get_elements in get_list:
		get_ids.append(get_elements[key_to_filter])
		for db_elemts in list_from_db:
			if db_elemts[key_to_filter] in get_ids:
				list_from_db.remove(db_elemts)
	return get_list +list_from_db